In [36]:
import pandas as pd
import numpy as np
import os
import json
import re
import time
import requests

In [46]:
file_path = './States/California/Sacramento/Sacramento_listings_results_grid_1_to_120.csv'
df = pd.read_csv(file_path)
df.columns

Index(['Room_id', 'Listing_url', 'Next_30_days_booked_days',
       'Next_30_to_60_days_booked_days', '75_rule_met', '55_rule_met',
       'Available_dates_by_year_and_month', 'Review_count_by_year_and_month',
       'Warning_level', 'Warning_type', 'Warning_message', 'Rating',
       'Accuracy_rating', 'Checking_rating', 'Cleanliness_rating',
       'Communication_rating', 'Location_rating', 'Value_rating',
       'Review_count', 'Review_months_this_year', 'Review_months_last_year',
       'Missing_review_months_this_year', 'Missing_review_months_last_year',
       'Total_missing_review_months_this_year',
       'Total_missing_review_months_last_year', 'Is_superhost', 'Guest_count',
       'Bedroom_count', 'Bed_count', 'Bath_count', 'Amenities', 'Co_hosts',
       'Highlights', 'Is_guest_favorite', 'Title', 'Latitude', 'Longitude',
       'Grid_index'],
      dtype='object')

In [47]:
print(f"Before sanitization: {df['Bath_count'].unique()}\n")

df.loc[df['Bath_count'] == 'Dedicated', 'Bath_count'] = 1
df.loc[df['Bath_count'] == 'Private', 'Bath_count'] = 1
df.loc[df['Bath_count'] == 'Shared', 'Bath_count'] = 1
df.loc[df['Bath_count'] == 'No', 'Bath_count'] = 0
df['Bath_count'] = df['Bath_count'].astype(float)

print(f"After sanitization: {sorted(df['Bath_count'].unique())}")

Before sanitization: ['1' '4' '2' '1.5' '2.5' '3' '3.5' 'Dedicated' 'Private' 'Shared' '4.5'
 'No' '0' '8' '5' '17']

After sanitization: [0.0, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 8.0, 17.0]


In [48]:
df.to_csv(file_path, index=False)
print(f"Sanitized file saved to {file_path}")

Sanitized file saved to ./States/California/Sacramento/Sacramento_listings_results_grid_1_to_120.csv
